This notebook can be used to converted the results from notebook 6 to segy files.

This method assumes that your model has the same dimensions as seismic data we have

In [ ]:
# IMPORTS
%load_ext autoreload
%autoreload 2
%matplotlib inline



import os
import numpy as np
import segyio

In [ ]:
dtype = np.float32

### Write inverted cube back to a segy

Not regularized:

In [ ]:
orig_file = '''.\SEISMIC\94p07ful.sgy'''

f = segyio.open(orig_file, ignore_geometry=True)
f.mmap()

False

In [ ]:
# Geometry reference

t = f.samples
il = f.attributes(segyio.TraceField.INLINE_3D)[:]
xl = f.attributes(segyio.TraceField.CROSSLINE_3D)[:]

traces = segyio.collect(f.trace)[:] # DATA
ntraces, nt = traces.shape # N traces and N time

# Regular IL and XL axes
il_unique = np.unique(il)
xl_unique = np.unique(xl)

il_min, il_max = min(il_unique), max(il_unique)
xl_min, xl_max = min(xl_unique), max(xl_unique)

dt = t[1] - t[0]
dil = min(np.unique(np.diff(il_unique)))
dxl = min(np.unique(np.diff(xl_unique)))

# regular axis
ilines = np.arange(il_min, il_max + dil, dil)
xlines = np.arange(xl_min, xl_max + dxl, dxl)
nil, nxl = ilines.size, xlines.size

ilgrid, xlgrid = np.meshgrid(np.arange(nil),
                             np.arange(nxl),
                             indexing='ij')

# look-up table
traces_indeces = np.full((nil, nxl), np.nan)
iils = (il - il_min) // dil
ixls = (xl - xl_min) // dxl
traces_indeces[iils, ixls] = np.arange(ntraces)
traces_available = np.logical_not(np.isnan(traces_indeces))
print('# missing traces: {}'.format(np.sum(~traces_available)))

# reorganize traces in regular grid
d = np.zeros((nil, nxl, nt))
d[ilgrid.ravel()[traces_available.ravel()],
  xlgrid.ravel()[traces_available.ravel()]] = traces

# missing traces: 0


In [ ]:
data_from_model = np.load("Your_exported_file.npy").astype(dtype)

In [ ]:
# Bring traces back to their original order and backuping
traces_to_save =  data_from_model[ilgrid.ravel()[traces_available.ravel()],
                                 xlgrid.ravel()[traces_available.ravel()]]

np.save("backup_tts.npy", traces_to_save)

To avoid time consuming rewriting of the geometry, we recommend copying preexisting file with correct geometries and substitution of traces individually.

In [ ]:
!cp ./SEISMIC/94p07ful.sgy ./inverted.segy

In [ ]:
itmin,itmax = 0, 0 # DEFINED BY YOUR WINDOW!

In [ ]:
segyfile_inverted = './inverted.segy'
inv_file = '''SEISMIC\Final_inversion_94p07_IPOPT.segy'''
with segyio.open(inv_file, ignore_geometry=True) as source:
        spec = segyio.spec()
        spec.format = int(source.format)
        spec.samples = t[itmin:itmax]
        spec.tracecount = source.tracecount
        print("spec done...")
        with segyio.open(segyfile_inverted, "r+") as destination:
            # Code below is commented as I use copied file of original data for destination
            # Copy all textual headers, including possible extended
            # for i in range(source.ext_headers):
            #     destination.text[i] = source.text[i]

            # # Copy the binary header, then insert the modifications needed for the new time axis
            # destination.bin = source.bin
            # destination.bin = {segyio.BinField.Samples: itmax-itmin}
            
            # # Copy all trace headers to destination file
            # destination.header = source.header 

            # Copy data and modify trace header
            from tqdm.notebook import tqdm
            for itrace in tqdm(range(destination.tracecount)):
                destination.header[itrace] = {segyio.TraceField.TRACE_SAMPLE_COUNT: itmax-itmin} 
                destination.trace[itrace] = traces_to_save[itrace].astype('float32')

In [ ]:
# Bring traces back to their original order and backuping

traces_to_save_reg =  m_relative_reg[ilgrid.ravel()[traces_available.ravel()],
                                 xlgrid.ravel()[traces_available.ravel()]]

np.save("backup_tts_reg.npy", traces_to_save_reg)

Saving Regularized inversion traces:

In [ ]:
itmin,itmax = 0, 0 # DEFINED BY YOUR WINDOW!

In [ ]:
segyfile_inverted_reg = './inverted_reg.segy'
with segyio.open(inv_file, ignore_geometry=True) as source:
        spec = segyio.spec()
        spec.format = int(source.format)
        spec.samples = t[itmin:itmax]
        spec.tracecount = source.tracecount
        print("spec done...")
        with segyio.open(segyfile_inverted_reg, "r+") as destination:
            # Code below is commented as I use copied file of original data for destination
            # Copy all textual headers, including possible extended
            # for i in range(source.ext_headers):
            #     destination.text[i] = source.text[i]

            # # Copy the binary header, then insert the modifications needed for the new time axis
            # destination.bin = source.bin
            # destination.bin = {segyio.BinField.Samples: itmax-itmin}
            
            # # Copy all trace headers to destination file
            # destination.header = source.header 

            # Copy data and modify trace header
            from tqdm.notebook import tqdm
            for itrace in tqdm(range(destination.tracecount)):
                destination.header[itrace] = {segyio.TraceField.TRACE_SAMPLE_COUNT: itmax-itmin} 
                destination.trace[itrace] = traces_to_save_reg[itrace].astype('float32')

Read the file back to check if saving was successful

In [ ]:
f1 = segyio.open(segyfile_inverted, ignore_geometry=True)
traces1 = segyio.collect(f1.trace)[:]

np.allclose(traces1, traces_to_save)
#Return True if writing was successful